In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
train_ds = torchvision.datasets.MNIST(
    root='../data',
    train=True,
    transform=transforms.ToTensor(),
    download=True
)
train_ds

Dataset MNIST
    Number of datapoints: 60000
    Root location: ../data
    Split: Train

In [3]:
test_ds = torchvision.datasets.MNIST(
    root='../data',
    train=False,
    transform=transforms.ToTensor()
)
test_ds

Dataset MNIST
    Number of datapoints: 10000
    Root location: ../data
    Split: Test

In [4]:
train_dl = torch.utils.data.DataLoader(train_ds,32,True)
test_dl = torch.utils.data.DataLoader(test_ds,32,False)

In [5]:
input_size = train_ds[0][0].numel()

In [6]:
model = nn.Linear(input_size,len(train_ds.classes))
model

Linear(in_features=784, out_features=10, bias=True)

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)

In [8]:
from tqdm import tqdm_notebook as tqdm

In [11]:
def one_epoch_man(model,train_dl,test_dl):
    model = model.cuda()
    for i,(images,labels) in enumerate(tqdm(train_dl)):
        model.train()
        images,labels = images.cuda(),labels.cuda()
        optimizer.zero_grad()

        preds = model(images.view(images.size(0),-1))

        loss = criterion(preds,labels)

        loss.backward()
        optimizer.step()
        if(i%500==0):
            correct = 0
            total = 0
            print(f'loss: {loss.item()}')
            for img,y in test_dl:
                model.eval()
                img,y = img.cuda(),y.cuda()
                preds = model(img.view(img.size(0),-1))
                preds = preds.argmax(1)
                total += preds.size(0)
                correct += (preds==y).sum()
            print(f'{correct} / {total}')

In [12]:
for i in range(10):
    one_epoch_man(model,train_dl,test_dl)

loss: 2.3753139972686768
1150 / 10000
loss: 0.41899314522743225
8925 / 10000
loss: 0.3912026882171631
9080 / 10000
loss: 0.42601221799850464
9141 / 10000



loss: 0.37266814708709717
9161 / 10000
loss: 0.18834368884563446
9185 / 10000
loss: 0.44317755103111267
9174 / 10000
loss: 0.3206779360771179
9188 / 10000



loss: 0.5764684081077576
9217 / 10000
loss: 0.18679891526699066
9231 / 10000
loss: 0.15329253673553467
9237 / 10000
loss: 0.40998196601867676
9237 / 10000



loss: 0.0951419323682785
9236 / 10000
loss: 0.13707146048545837
9250 / 10000


KeyboardInterrupt: 